In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_iiitb_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,"@IIITB_official Director, Prof Sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0,0,NaN,0.3400,Positive,4
1,1,May the peace and blessings of Christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"['Christmas', 'MerryChristmas', 'IIITB', 'IIIT...",IIITB_official,1,6,1.0,0.9337,Positive,5
2,2,Wishing everyone Merry Christmas & Happy holid...,Agroblock,1209470538738913280,2019-12-24 13:47:01,"['christmas', 'agritech', 'happyholidays', 'ii...",agroblock_io,0,1,NaN,0.9442,Positive,5
3,3,Our Ph.D. student Mr. Shyam S.M. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"['IIITB', 'IIITB10100pic']",IIITB_official,5,12,1.0,0.0000,Neutral,3
4,4,AgroBlock wishes Kisan Diwas to all the farmer...,Agroblock,1209072927921958912,2019-12-23 11:27:04,"['kisandiwas', 'farmersday', 'agritech', 'bloc...",agroblock_io,0,4,NaN,0.1531,Positive,3
5,5,Superb #Kohli #kohlidecadehttps://twitter.com/...,Harsh Dubey,1209069978479030272,2019-12-23 11:15:20,"['Kohli', 'kohlidecadehttps']",harsh_iiitb,0,0,NaN,0.6249,Positive,5
6,6,"Love your work, Veena! You’re the best ... @An...",Gerard Shanahan,1208283808434274304,2019-12-21 07:11:23,[],gas3040,0,2,NaN,0.8655,Positive,5
7,7,"Our last #GLE for 2019, in my home City Bangal...",Veena,1208184116614578176,2019-12-21 00:35:15,"['GLE', 'GLELeaders']",veena_haran,0,8,NaN,0.9246,Positive,5
8,8,But IIITB has 2000 students?,indian,1207827046862331904,2019-12-20 00:56:22,[],mkinthuk,0,0,NaN,0.0000,Neutral,3
9,9,"@IIITB_official Director, Prof Sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0,0,NaN,0.3400,Positive,4


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)

In [6]:
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,"@IIITB_official Director, Prof Sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,Positive,4,0.000000,0.000000
1,May the peace and blessings of Christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"['Christmas', 'MerryChristmas', 'IIITB', 'IIIT...",IIITB_official,1.0,6.0,1.0,0.9337,Positive,5,0.002294,0.000787
2,Wishing everyone Merry Christmas & Happy holid...,Agroblock,1209470538738913280,2019-12-24 13:47:01,"['christmas', 'agritech', 'happyholidays', 'ii...",agroblock_io,0.0,1.0,NaN,0.9442,Positive,5,0.000000,0.000131
3,Our Ph.D. student Mr. Shyam S.M. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"['IIITB', 'IIITB10100pic']",IIITB_official,5.0,12.0,1.0,0.0000,Neutral,3,0.011468,0.001574
4,AgroBlock wishes Kisan Diwas to all the farmer...,Agroblock,1209072927921958912,2019-12-23 11:27:04,"['kisandiwas', 'farmersday', 'agritech', 'bloc...",agroblock_io,0.0,4.0,NaN,0.1531,Positive,3,0.000000,0.000525
5,Superb #Kohli #kohlidecadehttps://twitter.com/...,Harsh Dubey,1209069978479030272,2019-12-23 11:15:20,"['Kohli', 'kohlidecadehttps']",harsh_iiitb,0.0,0.0,NaN,0.6249,Positive,5,0.000000,0.000000
6,"Love your work, Veena! You’re the best ... @An...",Gerard Shanahan,1208283808434274304,2019-12-21 07:11:23,[],gas3040,0.0,2.0,NaN,0.8655,Positive,5,0.000000,0.000262
7,"Our last #GLE for 2019, in my home City Bangal...",Veena,1208184116614578176,2019-12-21 00:35:15,"['GLE', 'GLELeaders']",veena_haran,0.0,8.0,NaN,0.9246,Positive,5,0.000000,0.001049
8,But IIITB has 2000 students?,indian,1207827046862331904,2019-12-20 00:56:22,[],mkinthuk,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000
9,"@IIITB_official Director, Prof Sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,Positive,4,0.000000,0.000000


In [7]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [8]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [9]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [10]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [11]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,"@iiitb_official director, prof sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,Positive,4,0.000000,0.000000,0.000000,@IIITB_official Director Prof Sadagopan welcom...,"[iiitb_official, director, prof, sadagopan, we..."
1,may the peace and blessings of christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"['Christmas', 'MerryChristmas', 'IIITB', 'IIIT...",IIITB_official,1.0,6.0,1.0,0.9337,Positive,5,0.002294,0.000787,0.169098,May the peace and blessings of Christmas be yo...,"[may, peace, blessings, christmas, may, coming..."
2,wishing everyone merry christmas & happy holid...,Agroblock,1209470538738913280,2019-12-24 13:47:01,"['christmas', 'agritech', 'happyholidays', 'ii...",agroblock_io,0.0,1.0,NaN,0.9442,Positive,5,0.000000,0.000131,0.005247,Wishing everyone Merry Christmas Happy holiday...,"[wishing, everyone, merry, christmas, happy, h..."
3,our ph.d. student mr. shyam s.m. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"['IIITB', 'IIITB10100pic']",IIITB_official,5.0,12.0,1.0,0.0000,Neutral,3,0.011468,0.001574,0.751041,Our Ph.D student Mr Shyam S.M has been selecte...,"[ph, student, mr, shyam, selected, prestigious..."
4,agroblock wishes kisan diwas to all the farmer...,Agroblock,1209072927921958912,2019-12-23 11:27:04,"['kisandiwas', 'farmersday', 'agritech', 'bloc...",agroblock_io,0.0,4.0,NaN,0.1531,Positive,3,0.000000,0.000525,0.020989,AgroBlock wishes Kisan Diwas to all the farmer...,"[agroblock, wishes, kisan, diwas, farmers, bac..."
5,superb #kohli #kohlidecadehttps://twitter.com/...,Harsh Dubey,1209069978479030272,2019-12-23 11:15:20,"['Kohli', 'kohlidecadehttps']",harsh_iiitb,0.0,0.0,NaN,0.6249,Positive,5,0.000000,0.000000,0.000000,Superb #Kohli # …,"[superb, kohli]"
6,"love your work, veena! you’re the best ... @an...",Gerard Shanahan,1208283808434274304,2019-12-21 07:11:23,[],gas3040,0.0,2.0,NaN,0.8655,Positive,5,0.000000,0.000262,0.010495,Love your work Veena You’re the best .. @AndyC...,"[love, work, veena, best, andycoxall, edherman..."
7,"our last #gle for 2019, in my home city bangal...",Veena,1208184116614578176,2019-12-21 00:35:15,"['GLE', 'GLELeaders']",veena_haran,0.0,8.0,NaN,0.9246,Positive,5,0.000000,0.001049,0.041978,Our last #GLE for 2019 in my home City Bangalo...,"[last, gle, 2019, home, city, bangalore, comes..."
8,but iiitb has 2000 students?,indian,1207827046862331904,2019-12-20 00:56:22,[],mkinthuk,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,But IIITB has 2000 students?,"[iiitb, 2000, students]"
9,"@iiitb_official director, prof sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,Positive,4,0.000000,0.000000,0.000000,@IIITB_official Director Prof Sadagopan welcom...,"[iiitb_official, director, prof, sadagopan, we..."


In [12]:
nltk.download('averaged_perceptron_tagger')
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Manam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,"@iiitb_official director, prof sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,Positive,4,0.000000,0.000000,0.000000,@IIITB_official Director Prof Sadagopan welcom...,"[iiitb_official, director, prof, sadagopan, we...","[(iiitb_official, JJ), (director, NN), (prof, ..."
1,may the peace and blessings of christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"['Christmas', 'MerryChristmas', 'IIITB', 'IIIT...",IIITB_official,1.0,6.0,1.0,0.9337,Positive,5,0.002294,0.000787,0.169098,May the peace and blessings of Christmas be yo...,"[may, peace, blessings, christmas, may, coming...","[(may, MD), (peace, VB), (blessings, NNS), (ch..."
2,wishing everyone merry christmas & happy holid...,Agroblock,1209470538738913280,2019-12-24 13:47:01,"['christmas', 'agritech', 'happyholidays', 'ii...",agroblock_io,0.0,1.0,NaN,0.9442,Positive,5,0.000000,0.000131,0.005247,Wishing everyone Merry Christmas Happy holiday...,"[wishing, everyone, merry, christmas, happy, h...","[(wishing, VBG), (everyone, NN), (merry, NN), ..."
3,our ph.d. student mr. shyam s.m. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"['IIITB', 'IIITB10100pic']",IIITB_official,5.0,12.0,1.0,0.0000,Neutral,3,0.011468,0.001574,0.751041,Our Ph.D student Mr Shyam S.M has been selecte...,"[ph, student, mr, shyam, selected, prestigious...","[(ph, JJ), (student, NN), (mr, NN), (shyam, NN..."
4,agroblock wishes kisan diwas to all the farmer...,Agroblock,1209072927921958912,2019-12-23 11:27:04,"['kisandiwas', 'farmersday', 'agritech', 'bloc...",agroblock_io,0.0,4.0,NaN,0.1531,Positive,3,0.000000,0.000525,0.020989,AgroBlock wishes Kisan Diwas to all the farmer...,"[agroblock, wishes, kisan, diwas, farmers, bac...","[(agroblock, NN), (wishes, NNS), (kisan, VBP),..."
5,superb #kohli #kohlidecadehttps://twitter.com/...,Harsh Dubey,1209069978479030272,2019-12-23 11:15:20,"['Kohli', 'kohlidecadehttps']",harsh_iiitb,0.0,0.0,NaN,0.6249,Positive,5,0.000000,0.000000,0.000000,Superb #Kohli # …,"[superb, kohli]","[(superb, NN), (kohli, NN)]"
6,"love your work, veena! you’re the best ... @an...",Gerard Shanahan,1208283808434274304,2019-12-21 07:11:23,[],gas3040,0.0,2.0,NaN,0.8655,Positive,5,0.000000,0.000262,0.010495,Love your work Veena You’re the best .. @AndyC...,"[love, work, veena, best, andycoxall, edherman...","[(love, NN), (work, NN), (veena, NN), (best, J..."
7,"our last #gle for 2019, in my home city bangal...",Veena,1208184116614578176,2019-12-21 00:35:15,"['GLE', 'GLELeaders']",veena_haran,0.0,8.0,NaN,0.9246,Positive,5,0.000000,0.001049,0.041978,Our last #GLE for 2019 in my home City Bangalo...,"[last, gle, 2019, home, city, bangalore, comes...","[(last, JJ), (gle, NN), (2019, CD), (home, NN)..."
8,but iiitb has 2000 students?,indian,1207827046862331904,2019-12-20 00:56:22,[],mkinthuk,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,But IIITB has 2000 students?,"[iiitb, 2000, students]","[(iiitb, NN), (2000, CD), (students, NNS)]"
9,"@iiitb_official director, prof sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,Positive,4,0.000000,0.000000,0.000000,@IIITB_official Director Prof Sadagopan welcom...,"[iiitb_official, director, prof, sadagopan, we...","[(iiitb_official, JJ), (director, NN), (prof, ..."


In [13]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,"@iiitb_official director, prof sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,Positive,4,0.000000,0.000000,0.000000,@IIITB_official Director Prof Sadagopan welcom...,"[iiitb_official, director, prof, sadagopan, we...","[(iiitb_official, JJ), (director, NN), (prof, ...","[director, prof, sadagopan, marjorie, vanbaeli..."
1,may the peace and blessings of christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"['Christmas', 'MerryChristmas', 'IIITB', 'IIIT...",IIITB_official,1.0,6.0,1.0,0.9337,Positive,5,0.002294,0.000787,0.169098,May the peace and blessings of Christmas be yo...,"[may, peace, blessings, christmas, may, coming...","[(may, MD), (peace, VB), (blessings, NNS), (ch...","[blessings, christmas, year, happiness, merry,..."
2,wishing everyone merry christmas & happy holid...,Agroblock,1209470538738913280,2019-12-24 13:47:01,"['christmas', 'agritech', 'happyholidays', 'ii...",agroblock_io,0.0,1.0,NaN,0.9442,Positive,5,0.000000,0.000131,0.005247,Wishing everyone Merry Christmas Happy holiday...,"[wishing, everyone, merry, christmas, happy, h...","[(wishing, VBG), (everyone, NN), (merry, NN), ...","[everyone, merry, holidays, year, features, ne..."
3,our ph.d. student mr. shyam s.m. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"['IIITB', 'IIITB10100pic']",IIITB_official,5.0,12.0,1.0,0.0000,Neutral,3,0.011468,0.001574,0.751041,Our Ph.D student Mr Shyam S.M has been selecte...,"[ph, student, mr, shyam, selected, prestigious...","[(ph, JJ), (student, NN), (mr, NN), (shyam, NN...","[student, mr, shyam, minister, fellowship, fic..."
4,agroblock wishes kisan diwas to all the farmer...,Agroblock,1209072927921958912,2019-12-23 11:27:04,"['kisandiwas', 'farmersday', 'agritech', 'bloc...",agroblock_io,0.0,4.0,NaN,0.1531,Positive,3,0.000000,0.000525,0.020989,AgroBlock wishes Kisan Diwas to all the farmer...,"[agroblock, wishes, kisan, diwas, farmers, bac...","[(agroblock, NN), (wishes, NNS), (kisan, VBP),...","[agroblock, wishes, farmers, country, lets, ch..."
5,superb #kohli #kohlidecadehttps://twitter.com/...,Harsh Dubey,1209069978479030272,2019-12-23 11:15:20,"['Kohli', 'kohlidecadehttps']",harsh_iiitb,0.0,0.0,NaN,0.6249,Positive,5,0.000000,0.000000,0.000000,Superb #Kohli # …,"[superb, kohli]","[(superb, NN), (kohli, NN)]","[superb, kohli]"
6,"love your work, veena! you’re the best ... @an...",Gerard Shanahan,1208283808434274304,2019-12-21 07:11:23,[],gas3040,0.0,2.0,NaN,0.8655,Positive,5,0.000000,0.000262,0.010495,Love your work Veena You’re the best .. @AndyC...,"[love, work, veena, best, andycoxall, edherman...","[(love, NN), (work, NN), (veena, NN), (best, J...","[love, work, veena, andycoxall, edhermanufc, c..."
7,"our last #gle for 2019, in my home city bangal...",Veena,1208184116614578176,2019-12-21 00:35:15,"['GLE', 'GLELeaders']",veena_haran,0.0,8.0,NaN,0.9246,Positive,5,0.000000,0.001049,0.041978,Our last #GLE for 2019 in my home City Bangalo...,"[last, gle, 2019, home, city, bangalore, comes...","[(last, JJ), (gle, NN), (2019, CD), (home, NN)...","[gle, home, city, bangalore, stories, students..."
8,but iiitb has 2000 students?,indian,1207827046862331904,2019-12-20 00:56:22,[],mkinthuk,0.0,0.0,NaN,0.0000,Neutral,3,0.000000,0.000000,0.000000,But IIITB has 2000 students?,"[iiitb, 2000, students]","[(iiitb, NN), (2000, CD), (students, NNS)]","[iiitb, students]"
9,"@iiitb_official director, prof sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,Positive,4,0.000000,0.000000,0.000000,@IIITB_official Director Prof Sadagopan welcom.

In [14]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,director
1,prof
2,sadagopan
3,marjorie
4,vanbaelinghem
5,consul
6,avenues
7,exchanges
8,affairs
9,studyinfrance


In [15]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,director,director,46
1,prof,prof,86
2,sadagopan,sadagopan,16
3,marjorie,marjorie,2
4,vanbaelinghem,vanbaelinghem,2
5,consul,consul,2
6,avenues,avenues,2
7,exchanges,exchanges,5
8,affairs,affairs,6
9,studyinfrance,studyinfrance,2


In [16]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
14,com,com,865
13,twitter,twitter,859
22,iiitb,iiitb,304
23,iiitb10100,iiitb10100,195
160,iiitbangalore,iiitbangalore,175
132,pic,pic,117
156,iiit,iiit,112
220,data,data,109
92,day,day,105
214,details,details,100


In [17]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [18]:
#list of irreleveant tweet topics
l=['weathercompany','skymapglobalpic','ssadagopan']
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
186,@iiitb_official hosting the leadership team of...,Karthik Kittu,1171375370785804288,2019-09-10 10:50:26,"['IIITB', 'innovation', 'Ai', 'MLpic']",karthikkittu,0.0,0.0,NaN,0.2732,Positive,4,0.000000,0.000000,0.000000,@IIITB_official hosting the leadership team of...,"[iiitb_official, hosting, leadership, team, na...","[(iiitb_official, JJ), (hosting, VBG), (leader...","[leadership, team, nasscom, president, debjani...",1
205,@iiitb_official hosting the leadership team of...,Karthik Kittu,1171375370785804288,2019-09-10 10:50:26,"['IIITB', 'innovation', 'Ai', 'MLpic']",karthikkittu,0.0,0.0,NaN,0.2732,Positive,4,0.000000,0.000000,0.000000,@IIITB_official hosting the leadership team of...,"[iiitb_official, hosting, leadership, team, na...","[(iiitb_official, JJ), (hosting, VBG), (leader...","[leadership, team, nasscom, president, debjani...",1
385,.@upgrad_edu and @iiitb_official mark their 3r...,Press Release Watch,1149917897352835072,2019-07-13 05:46:05,[],PrReleaseWatch,1.0,0.0,NaN,0.0000,Neutral,3,0.002294,0.000000,0.137615,.@upGrad_edu and @IIITB_official mark their 3r...,"[upgrad_edu, iiitb_official, mark, 3rd, gradua...","[(upgrad_edu, JJ), (iiitb_official, JJ), (mark...","[mark, graduation, ceremony, data, science, b,...",1
400,when a start up just shatters all expectations...,debjani ghosh,1173456720980373504,2019-09-16 04:40:58,"['VR', 'SobinThombra', 'HyperRealityTech']",debjani_ghosh_,1.0,13.0,NaN,0.9149,Positive,5,0.002294,0.001705,0.205829,When a start up just shatters all expectations...,"[start, shatters, expectations, vr, absolutely...","[(start, NN), (shatters, NNS), (expectations, ...","[start, shatters, expectations, work, sobintho...",1
401,had a fab interaction with @iiitb_official fac...,debjani ghosh,1173453993172799488,2019-09-16 04:30:08,[],debjani_ghosh_,1.0,14.0,NaN,0.9501,Positive,5,0.002294,0.001837,0.211077,Had a fab interaction with @IIITB_official fac...,"[fab, interaction, iiitb_official, faculty, am...","[(fab, JJ), (interaction, NN), (iiitb_official...","[interaction, faculty, super, depth, research,...",1
418,when a start up just shatters all expectations...,debjani ghosh,1173456720980373504,2019-09-16 04:40:58,"['VR', 'SobinThombra', 'HyperRealityTech']",debjani_ghosh_,1.0,13.0,NaN,0.9149,Positive,5,0.002294,0.001705,0.205829,When a start up just shatters all expectations...,"[start, shatters, expectations, vr, absolutely...","[(start, NN), (shatters, NNS), (expectations, ...","[start, shatters, expectations, work, sobintho...",1
419,had a fab interaction with @iiitb_official fac...,debjani ghosh,1173453993172799488,2019-09-16 04:30:08,[],debjani_ghosh_,1.0,14.0,NaN,0.9501,Positive,5,0.002294,0.001837,0.211077,Had a fab interaction with @IIITB_official fac...,"[fab, interaction, iiitb_official, faculty, am...","[(fab, JJ), (interaction, NN), (iiitb_official...","[interaction, faculty, super, depth, research,...",1
464,srinivasa ramanujan was one of the greatest ma...,Karthik Kittu,1189034182128717824,2019-10-29 04:20:14,"['Ramanujan', 'AI', 'ML', 'MachineLearningpic']",karthikkittu,2.0,3.0,NaN,0.6369,Positive,5,0.004587,0.000394,0.290971,Srinivasa Ramanujan was one of the greatest Ma...,"[srinivasa, ramanujan, one, greatest, mathemat...","[(srinivasa, NN), (ramanujan, VBZ), (one, CD),...","[srinivasa, mathematicians, time, gen, discove...",1
525,is there any registration process? kindly shar...,Startup🚀Sanatana(स्टार्टप्🚀सनातन),1198149364901761024,2019-11-23 08:00:43,[],StartupSanatana,0.0,0.0,NaN,0.8240,Positive,5,0.000000,0.000000,0.000000,Is there any Registration Process Kindly Share...,"[registration, process, kindly, share, link, u...","[(registration, NN), (process, NN), (kindly, R...","[registration, process, share, register, thx, ...",1
528,"prof @ssadagopan hosted maharaja of mysuru,....",Uday B

In [19]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [20]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [21]:
df['new_score']=df['score']

In [22]:
df_eng1=df[df.engagement==1]

In [23]:
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
df[df.engagement==1]=df_eng1

In [25]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [26]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [27]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,"@iiitb_official director, prof sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,...,0.000000,0.000000,0.000000,@IIITB_official Director Prof Sadagopan welcom...,"[iiitb_official, director, prof, sadagopan, we...","[(iiitb_official, JJ), (director, NN), (prof, ...","[director, prof, sadagopan, marjorie, vanbaeli...",0,0,4
1,may the peace and blessings of christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"['Christmas', 'MerryChristmas', 'IIITB', 'IIIT...",IIITB_official,1.0,6.0,1.0,0.9337,...,0.002294,0.000787,0.169098,May the peace and blessings of Christmas be yo...,"[may, peace, blessings, christmas, may, coming...","[(may, MD), (peace, VB), (blessings, NNS), (ch...","[blessings, christmas, year, happiness, merry,...",0,1,3
2,wishing everyone merry christmas & happy holid...,Agroblock,1209470538738913280,2019-12-24 13:47:01,"['christmas', 'agritech', 'happyholidays', 'ii...",agroblock_io,0.0,1.0,NaN,0.9442,...,0.000000,0.000131,0.005247,Wishing everyone Merry Christmas Happy holiday...,"[wishing, everyone, merry, christmas, happy, h...","[(wishing, VBG), (everyone, NN), (merry, NN), ...","[everyone, merry, holidays, year, features, ne...",0,0,5
3,our ph.d. student mr. shyam s.m. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"['IIITB', 'IIITB10100pic']",IIITB_official,5.0,12.0,1.0,0.0000,...,0.011468,0.001574,0.751041,Our Ph.D student Mr Shyam S.M has been selecte...,"[ph, student, mr, shyam, selected, prestigious...","[(ph, JJ), (student, NN), (mr, NN), (shyam, NN...","[student, mr, shyam, minister, fellowship, fic...",0,1,3
4,agroblock wishes kisan diwas to all the farmer...,Agroblock,1209072927921958912,2019-12-23 11:27:04,"['kisandiwas', 'farmersday', 'agritech', 'bloc...",agroblock_io,0.0,4.0,NaN,0.1531,...,0.000000,0.000525,0.020989,AgroBlock wishes Kisan Diwas to all the farmer...,"[agroblock, wishes, kisan, diwas, farmers, bac...","[(agroblock, NN), (wishes, NNS), (kisan, VBP),...","[agroblock, wishes, farmers, country, lets, ch...",0,1,3
5,superb #kohli #kohlidecadehttps://twitter.com/...,Harsh Dubey,1209069978479030272,2019-12-23 11:15:20,"['Kohli', 'kohlidecadehttps']",harsh_iiitb,0.0,0.0,NaN,0.6249,...,0.000000,0.000000,0.000000,Superb #Kohli # …,"[superb, kohli]","[(superb, NN), (kohli, NN)]","[superb, kohli]",0,0,5
6,"love your work, veena! you’re the best ... @an...",Gerard Shanahan,1208283808434274304,2019-12-21 07:11:23,[],gas3040,0.0,2.0,NaN,0.8655,...,0.000000,0.000262,0.010495,Love your work Veena You’re the best .. @AndyC...,"[love, work, veena, best, andycoxall, edherman...","[(love, NN), (work, NN), (veena, NN), (best, J...","[love, work, veena, andycoxall, edhermanufc, c...",0,0,5
7,"our last #gle for 2019, in my home city bangal...",Veena,1208184116614578176,2019-12-21 00:35:15,"['GLE', 'GLELeaders']",veena_haran,0.0,8.0,NaN,0.9246,...,0.000000,0.001049,0.041978,Our last #GLE for 2019 in my home City Bangalo...,"[last, gle, 2019, home, city, bangalore, comes...","[(last, JJ), (gle, NN), (2019, CD), (home, NN)...","[gle, home, city, bangalore, stories, students...",0,1,5
8,but iiitb has 2000 students?,indian,1207827046862331904,2019-12-20 00:56:22,[],mkinthuk,0.0,0.0,NaN,0.0000,...,0.000000,0.000000,0.000000,But IIITB has 2000 students?,"[iiitb, 2000, students]","[(iiitb, NN), (2000, CD), (students, NNS)]","[iiitb, students]",0,0,3
9,"@iiitb_official director, prof sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"['collaboration', 'academic', 'exchanges', 'st...",BangaloreFrance,0.0,0.0,NaN,0.3400,...,0.000000,0.000000,0.000000,@IIITB_official Director Prof Sada

In [28]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/IIITBWeightedSentiment.csv')